# 🎨 Model Merger Notebook

This notebook contains examples of all the different model merging techniques available in Chattiori Model Merger, with explanations and ready-to-run code.

## 📁 Setup Variables

First, let's import the merge module and set up our paths and model variables.

In [6]:
import subprocess
import os
import datetime
import threading
import queue

# Path configuration
models_path = "../../ComfyUI/models/checkpoints"
vae_path = "../../ComfyUI/models/vae"

# Checkpoint Model variables
model_1 = "plantMilkModelSuite_walnut.safetensors"
model_2 = "grandDesignILXL20_v1.safetensors"
model_3 = "ilustmix_v55.safetensors"
model_4 = "ilustmix_v6.safetensors"
model_5 = "ilustmix_v9.safetensors"

# Vae Model variables
vae_1 = "anything-v4.0.vae.pt"

# Vae selection
vae_model = vae_1

# Use the correct Python executable from virtual environment
python_exe = "C:/Projects/Chattiori-Model-Merger/venv/Scripts/python.exe"

def clear_merge_log():
    """Clear the last_merge.log file at the start of each run"""
    try:
        with open("last_merge.log", "w", encoding="utf-8") as f:
            f.write(f"=== Merge Log Started at {datetime.datetime.now()} ===\n\n")
    except Exception:
        pass

def log_error(error_type, message, is_blocking=False):
    """Log errors to file and display clean message to user"""
    status = "🚫 Blocking" if is_blocking else "⚠️ Non-blocking"
    
    # Write detailed error to log file
    try:
        with open("last_merge.log", "a", encoding="utf-8") as f:
            f.write(f"[{datetime.datetime.now()}] {status}: {error_type}\n")
            f.write(f"Details: {message}\n")
            f.write("-" * 50 + "\n\n")
    except Exception:
        pass
    
    # Show clean message to user
    print(f"{status} {error_type}: {message.split(':')[-1].strip()}")
    print("📝 See last_merge.log for more details.")

def stream_output_safely(process, output_queue):
    """Safely stream subprocess output, handling encoding issues"""
    try:
        # Try UTF-8 first, then fall back to system encoding with error handling
        for line in process.stdout:
            try:
                decoded_line = line.decode('utf-8')
            except UnicodeDecodeError:
                try:
                    decoded_line = line.decode('cp1252', errors='replace')
                except UnicodeDecodeError:
                    decoded_line = line.decode('utf-8', errors='replace')
            output_queue.put(('stdout', decoded_line))
    except Exception as e:
        log_error("UnicodeDecodeError", str(e), is_blocking=False)
    finally:
        output_queue.put(('done', None))

# Helper function to run merge commands
def run_merge(mode, model0, model1, model2=None, alpha=0.5, beta=0.5, output_name="merged", **kwargs):
    """
    Run a merge operation with the specified parameters
    """
    # Clear log at start of each merge
    clear_merge_log()
    
    cmd = [
        python_exe, "lib/merge.py",  # Updated path to lib directory
        mode, models_path, model0, model1
    ]
    
    # Add model_2 if provided
    if model2:
        cmd.extend(["--model_2", model2])
    
    # Add alpha and beta
    cmd.extend(["--alpha", str(alpha)])
    if beta != 0.5 or mode in ["TRS", "ST", "TS", "SIM", "MD", "DARE"]:
        cmd.extend(["--beta", str(beta)])
    
    # Add VAE path
    cmd.extend(["--vae", f"{vae_path}/{vae_model}"])
    
    # Add standard flags
    cmd.extend(["--prune", "--save_half", "--save_safetensors"])
    
    # Default to CUDA device for GPU acceleration (can be overridden)
    if "device" not in kwargs:
        kwargs["device"] = "cuda"
    
    # Add output name
    cmd.extend(["--output", output_name])
    
    # Add any additional kwargs
    for key, value in kwargs.items():
        if isinstance(value, bool) and value:
            cmd.append(f"--{key}")
        elif not isinstance(value, bool):
            cmd.extend([f"--{key}", str(value)])
    
    print("🚀 Starting merge:", " ".join(cmd))
    
    try:
        # Use Popen for better output handling
        process = subprocess.Popen(
            cmd, 
            stdout=subprocess.PIPE, 
            stderr=subprocess.STDOUT,  # Combine stderr with stdout
            text=False  # Handle encoding ourselves
        )
        
        # Create a queue for thread-safe communication
        output_queue = queue.Queue()
        
        # Start output streaming thread
        output_thread = threading.Thread(
            target=stream_output_safely, 
            args=(process, output_queue)
        )
        output_thread.daemon = True
        output_thread.start()
        
        # Collect all output
        all_output = []
        while True:
            try:
                msg_type, content = output_queue.get(timeout=1)
                if msg_type == 'done':
                    break
                elif msg_type == 'stdout':
                    all_output.append(content)
                    print(content, end='')  # Real-time output
            except queue.Empty:
                # Check if process is still running
                if process.poll() is not None:
                    break
        
        # Wait for process to complete
        return_code = process.wait()
        output_thread.join(timeout=1)
        
        combined_output = ''.join(all_output)
        
        if return_code == 0:
            print("\n✅ Merge completed successfully!")
        else:
            print(f"\n🚫 Merge failed (exit code: {return_code})")
            log_error("ProcessError", f"Merge process failed with exit code {return_code}", is_blocking=True)
            
        # Log the full output for reference
        try:
            with open("last_merge.log", "a", encoding="utf-8") as f:
                f.write("=== Full Process Output ===\n")
                f.write(combined_output)
                f.write("\n=== End Output ===\n")
        except Exception:
            pass
            
        return type('Result', (), {
            'returncode': return_code,
            'stdout': combined_output,
            'stderr': ''
        })()
        
    except Exception as e:
        log_error("ProcessStartError", str(e), is_blocking=True)
        print(f"🚫 Failed to start merge process: {e}")
        return type('Result', (), {
            'returncode': 1,
            'stdout': '',
            'stderr': str(e)
        })()

## 1️⃣ **WS - Weighted Sum (2 Models)**

### 🛠️ How It Works

- Directly blends **two models** at a specified alpha.

### ✅ When to Use

- When you want a **simple, balanced mix**.

- Ideal for merging **two models of similar style or purpose**.

- Great for combining **two complementary styles (like realism and anime)**.

### ⚠️ Why Use It

- **Fast, simple, predictable.**

- Best when you want **even mixing of features** without overcomplication.

In [5]:
# Example: Weighted Sum merge
result = run_merge(
    mode="WS",
    model0=model_1,
    model1=model_2,
    alpha=0.4,
    output_name="WS_example"
)

Running command: C:/Projects/Chattiori-Model-Merger/venv/Scripts/python.exe merge.py WS ../../ComfyUI/models/checkpoints plantMilkModelSuite_walnut.safetensors grandDesignILXL20_v1.safetensors --alpha 0.4 --vae ../../ComfyUI/models/vae/anything-v4.0.vae.pt --prune --save_half --save_safetensors --output WS_example --device cuda


Exception in thread Thread-8 (_readerthread):
Traceback (most recent call last):
  File "c:\Libraries\Python\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\Czarena\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "c:\Libraries\Python\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Libraries\Python\Lib\subprocess.py", line 1601, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "c:\Libraries\Python\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8d in position 1528: character maps to <undefined>


✅ Merge completed successfully!
Loading plantMilkModelSuite_walnut...
Loading grandDesignILXL20_v1...
alpha weight converted for XL[0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0]
Saving as WS_example.safetensors...
Done! (6.46G)



## 2️⃣ **SIG - Sigmoid Merge (2 Models)**

### 🛠️ How It Works

- Uses a **sigmoid curve function** to blend models.
- It **favors details from the stronger model** more than Weighted Sum.

### ✅ When to Use

- If you want to preserve more detail from **one dominant model**, but still have a **touch** of the other model.
- Great for preserving **fine textures or shading from one source**.

### ⚠️ Why Use It

- Provides a more **organic blend**.
- Useful when one model is **higher quality or more detailed**, and you don't want to lose that.

In [ ]:
# Sigmoid merge
result = run_merge(
    mode="SIG",
    model0=model_1,
    model1=model_2,
    alpha=0.4,
    output_name="SIG_example"
)

## 3️⃣ **GEO - Geometric Merge (2 Models)**

### 🛠️ How It Works

- Multiplies the latent spaces of the models, which often **smooths out artifacts** and enhances compatibility.
- Often results in a **cleaner but less creative blend**.

### ✅ When to Use

- When you want **more structural stability**.
- Useful if you're merging **two very different models**, such as realism + anime.

### ⚠️ Why Use It

- Reduces **glitches and noise** from mismatched styles.
- Keeps **core structure stable**, even if styles differ.

In [ ]:
# Geometric merge
result = run_merge(
    mode="GEO",
    model0=model_1,
    model1=model_2,
    alpha=0.4,
    output_name="GEO_example"
)

## 4️⃣ **MAX - Maximum Merge (2 Models)**

### 🛠️ How It Works

- Picks the **maximum value from each model's weights**.
- This strongly favors **the stronger model** in each part of the latent space.

### ✅ When to Use

- When you want to **preserve maximum detail from both models**.
- Can be **unstable**, but great for maximizing detail.

### ⚠️ Why Use It

- Best when working with **high-detail models**.
- Useful for **testing upper limits** of merges.

In [ ]:
# Maximum merge
result = run_merge(
    mode="MAX",
    model0=model_1,
    model1=model_2,
    alpha=0.4,
    output_name="MAX_example"
)

## 5️⃣ **AD - Add Difference (3 Models)**

### 🛠️ How It Works

- Uses **Model 3** as a "difference model" to guide the blend between **Model 1** and **Model 2**.
- This lets you **add features from Model 3 without fully blending it**.

### ✅ When to Use

- When you have a **base blend (Model 1 + Model 2)** and want to **inject features from Model 3**.
- Example: Combine two anime models, but add **realism shading from Model 3**.

### ⚠️ Why Use It

- Preserves control over the core blend while still adding new features.
- One of the most **flexible triple merges**.

In [ ]:
# Add Difference merge (3 models)
result = run_merge(
    mode="AD",
    model0=model_1,
    model1=model_2,
    model2=model_3,
    alpha=0.4,
    output_name="AD_example"
)

## 6️⃣ **sAD - Smooth Add Difference (3 Models)**

### 🛠️ How It Works

- Similar to **AD**, but with smoother interpolation between models.
- It softens the impact of Model 3.

### ✅ When to Use

- When Model 3 has **very different features**, and you want to **soften its influence**.
- Best for introducing **stylistic tweaks without overpowering the blend**.

### ⚠️ Why Use It

- Good for subtle refinements in **texture or color**.

In [ ]:
# Smooth Add Difference merge (3 models)
result = run_merge(
    mode="sAD",
    model0=model_1,
    model1=model_2,
    model2=model_3,
    alpha=0.55,
    beta=0.35,
    output_name="sAD_example"
)

## 7️⃣ **MD - Multiply Difference (3 Models + Beta)**

### 🛠️ How It Works

- Multiplies the differences between **Model 1 and 2**, then adds them to **Model 3**.
- **Beta** controls how strongly Model 3 influences the final mix.

### ✅ When to Use

- When Model 3 acts as a **refinement base**, and you want to **multiply stylistic differences** into it.
- Example: Apply the stylistic contrast between **anime and realism models** into a **semi-realistic base model**.

### ⚠️ Why Use It

- Very creative but hard to predict.
- Ideal for **experimental merges**.

In [ ]:
# Multiply Difference merge (3 models with beta)
result = run_merge(
    mode="MD",
    model0=model_1,
    model1=model_2,
    model2=model_3,
    alpha=0.5,
    beta=0.3,
    output_name="MD_example"
)

## 8️⃣ **TRS - Triple Sum (3 Models + Beta)**

### 🛠️ How It Works

- Averages 3 models, with **beta controlling how much Model 3 contributes**.
- Simple 3-way average.

### ✅ When to Use

- When you want a **balanced, even blend** between all 3 models.
- Good for combining **three models of equal quality**.

### ⚠️ Why Use It

- Simple and reliable if you want an **even mix of all sources**.

In [ ]:
# Triple Sum merge (3 models with beta)
result = run_merge(
    mode="TRS",
    model0=model_1,
    model1=model_2,
    model2=model_3,
    alpha=0.33,
    beta=0.33,
    output_name="TRS_example"
)

## 9️⃣ **DARE - DARE Merge (2 Models)**

### 🛠️ How It Works

- Applies **advanced blending techniques** combining 2 models.
- Designed for **experimental blending**.

### ✅ When to Use

- When you want to **smooth out the final step of a merge**.
- Can produce unexpected but unique results.

In [ ]:
# DARE merge (experimental blending)
result = run_merge(
    mode="DARE",
    model0=model_1,
    model1=model_2,
    alpha=0.4,
    beta=0.3,  # DARE requires beta parameter
    output_name="DARE_example"
)

## 💡 **Summary & Tips**

### 📝 **Quick Reference Guide**

✅ **For stability and quality:** Use **WS**, **GEO**, or **SIG**.  
✅ **For artistic fusion:** Use **AD**, **sAD**, or **TD**.  
✅ **For experimental creativity:** Use **MD**, **DARE**, or **SIM**.

### 🔧 **Key Parameters**

- **Alpha**: Controls the ratio between model 0 and model 1
  - `0.3` = 70% model 0, 30% model 1
  - `0.7` = 30% model 0, 70% model 1
  
- **Beta**: Controls how much model 2 influences the mix (for 3-model merges)
  - `0.35` = 65% (model 0 + model 1), 35% model 2

### ⚙️ **Standard Flags Used**

- `--prune`: Strips models of original VAE and metadata (always recommended)
- `--save_half`: Saves as FP16 format (most common)
- `--save_safetensors`: Saves as .safetensors format (recommended)
- `--vae`: Adds SDXL VAE to the merged model

### 🚩 **Complete Flag Reference**

#### **Boolean Flags** (Use as `flag_name=True`)

- `force=True` - **Force overwrite** existing output files
- `delete_source=True` - **Delete source** checkpoint files after merging (keeps VAE)
- `no_metadata=True` - **Save without metadata** (smaller file size)
- `keep_ema=True` - **Keep EMA weights** in the output
- `save_quarter=True` - **Save as float8** (experimental, smaller file)

#### **Model Structure Flags** (Choose only one)

- `cosine0=True` - **Favor model 0's structure** with details from others
- `cosine1=True` - **Favor model 1's structure** with details from others  
- `cosine2=True` - **Favor model 2's structure** with details from others

#### **Model Difference Flags** (Advanced users)

- `use_dif_10=True` - Use **difference between model 0 and 1** as model 1
- `use_dif_20=True` - Use **difference between model 0 and 2** as model 2
- `use_dif_21=True` - Use **difference between model 2 and 1** as model 2

#### **Value Flags** (Use as `flag_name="value"` or `flag_name=number`)

- `seed=12345` - **Random seed** for stochastic modes (DARE, etc.)
- `memo="My merge description"` - **Add description** to metadata
- `device="cuda"` - **Use GPU** instead of CPU (much faster!)
- `fine="key_name"` - **Finetune specific keys** on model 0
- `m0_name="Custom Model 0"` - **Custom name** for model 0 in metadata
- `m1_name="Custom Model 1"` - **Custom name** for model 1 in metadata
- `m2_name="Custom Model 2"` - **Custom name** for model 2 in metadata

#### **Random Parameter Flags** (Advanced)

- `rand_alpha="0.2,0.8,42"` - **Randomize alpha** (min, max, seed)
- `rand_beta="0.1,0.5,123"` - **Randomize beta** (min, max, seed)

### 💡 **Flag Usage Examples**

```python
# Basic merge with force overwrite
run_merge("WS", model_1, model_2, alpha=0.4, force=True)

# GPU-accelerated merge with custom metadata
run_merge("SIG", model_1, model_2, alpha=0.6, 
          device="cuda", memo="High quality sigmoid blend")

# Advanced 3-model merge favoring first model's structure
run_merge("sAD", model_1, model_2, model_3, alpha=0.5, beta=0.3,
          cosine0=True, seed=42)

# Experimental merge with cleanup
run_merge("DARE", model_1, model_2, alpha=0.4, beta=0.3,
          delete_source=True, no_metadata=True)
```

### 🎯 **Pro Tips**

1. Start with **WS** or **SIG** for your first merges - they're predictable and stable
2. Use **GEO** when merging very different styles (like anime + realism)
3. **AD** and **sAD** are great for adding specific features from a third model
4. Always backup your models before experimenting!
5. Try different alpha/beta values - small changes can make big differences
6. Use `device="cuda"` if you have a GPU - it's **much faster**!
7. The `cosine` flags are powerful for preserving one model's structure